# Beagle

## Setup

In [1]:
from src.embedder import Embedder
from src.search_engine import SearchEngine
from src.database_manager import DatabaseManager
from src.content_manager import ContentManager

# Don't be fooled by the name. The embeddings are float16 tensors.
embedder = Embedder("ViT-B/32")
database_manager = DatabaseManager()

engine = SearchEngine(database_manager)

content_manager = ContentManager(database_manager, embedder)

content_manager.process_input_file("scope.txt")

# Querying the database

In [2]:
query_embedding = embedder.embedText("Army")

for file_path, similarity in engine.search_for(query_embedding):
    print(f"File {file_path} has a similarity of {similarity[0]: .2f} to your query")

File res/1.png has a similarity of  0.24 to your query
File res/6.png has a similarity of  0.24 to your query
File res/4.png has a similarity of  0.23 to your query
File res/3.png has a similarity of  0.21 to your query
File res/2.png has a similarity of  0.19 to your query
File res/5.png has a similarity of  0.19 to your query
File res/7.png has a similarity of  0.17 to your query
